In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk                                         #Natural language processing tool-kit
from nltk.corpus import stopwords                   #Stopwords corpus

import re

# %% [code]

def processing (train):
    train['Comments']=train['Comments'].astype(str) #converting comments to string
    #Stopwords
    stop = set(stopwords.words('english')) 
    #Removing Stopwords
    train['Comments'] = train['Comments'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    #Lowercase
    train['Comments'] = train['Comments'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    #punctuations removing
    train['Comments']= train['Comments'].str.replace('[!”#$%&’()*+,-./:;<=>?[\]^_`{|}~]','')
    #numbers removing
    train['Comments']= train['Comments'].str.replace('[1234567890]','')
    #finding top 5 most common words
    Most_common_word = pd.Series(' '.join(train['Comments']).split()).value_counts()[:5]
    #finding top 20 most rare words
    rare_word = pd.Series(' '.join(train['Comments']).split()).value_counts()[-20:]
    #Removing Common &rare words
    train['Comments'] =train['Comments'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_word))
    train['Comments'] =train['Comments'].apply(lambda x: " ".join(x for x in x.split() if x not in Most_common_word))
    #data cleaning
    train['Comments'] = train['Comments'].apply(lambda x : re.sub(r"(.)\1{2,}", r"\1\1\1", x))
    train['Comments'] = train['Comments'].apply(lambda x : re.sub(r"([a-z],2)\1{2,}", r"\1\1\1", x))
    train['Comments'] = train['Comments'].apply(lambda x : re.sub(r"@\w+", r"@username", x))
    
    lemm=[]
    # Lemmatize with POS Tag
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0]
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
        
        return tag_dict.get(tag, wordnet.NOUN)
        
        
    lemmatizer = WordNetLemmatizer()
    for sentence in train['Comments']:
        words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
        lemm.append(words)
    train['Comments'] = lemm
    final = []
    for row in train['Comments']:
        sequ = ''
        for word in row:
            sequ = sequ + ' ' + word
        final.append(sequ)
    train['Comments'] = final
    #Stemming
    stemm = []
    nltsnow = nltk.stem.SnowballStemmer('english')
    for sentence in train['Comments']:
        words = [nltsnow.stem(word) for word in sentence.split()]   
        stemm.append(words)
    train['Comments'] = stemm
    
    finalprocessing = []
    for row in train['Comments']:
        sequ = ''
        for word in row:
            sequ = sequ + ' ' + word
        finalprocessing.append(sequ)
    train['Comments'] = finalprocessing
    
    train = train.drop(["UserIndex", "Number of Comments", "Number of Subscribers", "Membership Duration", "Number of Uploads", "Profanity in UserID", "Age"], axis = 1)
    return (train)